In [1]:

"""
@author: Mohammad Asif Zaman
@coathors: García Lagos Miriam, Núñez Vega Dulce Angelica,  Sánchez Montalvo Luis Antonio
"""
# Usando el sistema de unidades de Planck (hcut = c = 1)
# Ref.: http://newfi.narod.ru/Newfi/Constants.htm
using Plots

hc = 197.327                # Factor de Conversion  MeV fm (hut * c)
G = hc * 6.67259e-45        # Constante Gravitacional
Ms = 1.1157467e60
rho_s = 1665.3              # Funcion de Densidad Central (Densidad con r = 0)
M0 = (4*3.14159265*(G^3)*rho_s)^(-0.5)
R0 = G*M0
mn = 938.926                # Masa de un neutron con MeV c^-2

# Se definen las funciones

# Función para encontrar el valor inicial de n(r=0)
function initial_n()         
    n = 1
    err = 1
    tol = 1e-15
    count = 0
    # Método de Newton-Raphson 
    while err > tol  
        count += 1
        fn = n*mn + 236*n^(2.54) - rho_s
        dfn = mn + 236*2.54*n^(1.54)
        temp = n - fn/dfn
        err = abs(n-temp)
        n = temp
    end
    println("El metodo de Newton-Raphson converge a las ", count, " iteraciones")
    return n
end

function rho(p)  #Densidad
    n = (p*rho_s/363.44)^(1/2.54)
    return (236 * n^2.54 + n *mn)/rho_s 
end
    

function dp_dr(r,m,p,flag)
    if flag == 1                              # Modelo Newtoniano 
        y = -m*rho(p)/(r^2 + 1e-20)
    else                                      # Modelo Relativista
        rh = rho(p)                            
        y = -(p+rh)*(p*r^3 + m)/(r^2 - 2*m*r + 1e-20)
    end
    return y
end
    

function dm_dr(r,m,p) 
    return rho(p)*r^2
end

dm_dr (generic function with 1 method)

In [2]:
#Método de Runge Kutta de Orden 4
    
function RK4Solver(r,m,p,h,flag)
    y = zeros(2)
    k11 = dm_dr(r,m,p)
    k21 = dp_dr(r,m,p,flag)
    
    k12 = dm_dr(r+0.5*h,m+0.5*k11*h,p+0.5*k21*h)
    k22 = dp_dr(r+0.5*h,m+0.5*k11*h,p+0.5*k21*h,flag)
    
    k13 = dm_dr(r+0.5*h,m+0.5*k12*h,p+0.5*k22*h)
    k23 = dp_dr(r+0.5*h,m+0.5*k12*h,p+0.5*k22*h,flag)
    
    k14 = dm_dr(r+h,m+h*k13,p+h*k23)    
    k24 = dp_dr(r+h,m+h*k13,p+h*k23,flag)    
    
    y[1] = m + h*(k11 + 2*k12 + 2*k13 + k14)/6.
    y[2] = p + h*(k21 + 2*k22 + 2*k23 + k24)/6.
    return y[1], y[2]
end
    
    
function mplot(x,y,xl,yl,clr,lbl)
    
    return plot(x,y,xlabel=xl,ylabel=yl,  color=clr,label = lbl)
end

mplot (generic function with 1 method)

In [3]:
#Codigo de la simulación

#Definimos los datos que nos seran de utilidad.

N = 4000
r = collect(LinRange(0,15,N))
h = r[2]-r[1]

m = zeros(N)
p = zeros(N)
rh = zeros(N)
ni = initial_n()

r[1]= 0
m[1]= 0
p[1]= 363.44 * (ni^2.54)/rho_s
rh[1]= 1

flag_set = [1,2]
println("Densidad inicial, ni = ", ni)
println("Presión inicial, P[0] = ", p[1])

println("Rango de simulación, R = 0 a ", r[end]*R0*1e-18, " km") 
tol = 9e-5

# Iteramos sobre los 2 modelos (k = 1: Clasico, k = 2: Relativista)
for k in 1:2
    flag = flag_set[k]
    println("Corriendo RK4")
    for i in 1:N-1
        m[i+1], p[i+1] = RK4Solver(r[i],m[i],p[i],h,flag)
        rh[i+1] = rho(r[i])
        if p[i+1] < tol
            rf = r[i]
            mf = m[i]
            if i == N-1
                println("El programa no comverge con P=0, extienda el maximo valor de r")
            else
                println("P < ", tol, " despues de ", i, " iteraciones")
            end
            println(' ')
            m = m[1:i+1]
            p = p[1:i+1]
            rh = rh[1:i+1]
            r = r[1:i+1]
            if flag == 1
                lbl ="Modelo Newtoniano"
                clr ="blue"
            else
                lbl ="Modelo Relativista"
                clr ="red"
            end
            println("==============================================")
            println(lbl, " Resultados")
            println("==============================================")
            println("Densidad inicial, rho_s = ", rho_s, " MeV/fm3")
            println("Masa total = ", mf*M0/Ms, " veces la masa Solar")
            println("Radio de la Estrella de Neutrones= ", rf*R0*1e-18, " km")


            mplot(r*R0*1e-18,p*rho_s,"Distance, r(km)","Pressure MeVfm^-3 ", clr, lbl)
            #plot!(r*R0*1e-18,p*rho_s,xlabel="Distance, r(km)",ylabel="Pressure MeVfm^-3",  color=clr,label = lbl)
            mplot(r*R0*1e-18,m*M0/Ms,"Distance, r(km)","Mass M/M_sun ", clr , lbl )
            #plot!(r*R0*1e-18,m*M0/Ms,xlabel="Distance, r(km)",ylabel="Mass M/M_sun",  color=clr,label = lbl)
            break
        end
    end
    
end
    
    
    

El metodo de Newton-Raphson converge a las 5 iteraciones
Densidad inicial, ni = 1.2918969375342138
Presión inicial, P[0] = 0.4182722266764404
Rango de simulación, R = 0 a 90.36486611870907 km
Corriendo RK4
P < 9.0e-5 despues de 734 iteraciones
 
Modelo Newtoniano Resultados
Densidad inicial, rho_s = 1665.3 MeV/fm3
Masa total = 10.068235234651203 veces la masa Solar
Radio de la Estrella de Neutrones= 16.563502591901415 km
Corriendo RK4
P < 9.0e-5 despues de 431 iteraciones
 
Modelo Relativista Resultados
Densidad inicial, rho_s = 1665.3 MeV/fm3
Masa total = 1.876502808441437 veces la masa Solar
Radio de la Estrella de Neutrones= 9.716652270828934 km
